In [1]:
import psycopg2
import pandas as pd
from datetime import date, timedelta

In [2]:
conn = psycopg2.connect(database="training_2015", 
                        user="rasch", 
                        password="", 
                        host="dssgsummer2014postgres.c5faqozfo86k.us-west-2.rds.amazonaws.com", 
                        port="5432")

def query_db(sql):
    return pd.read_sql(sql, conn)

### For each location, how many failed inspections where there since April 2015?

In [22]:
sql_query = """SELECT COUNT(*), "LOCATION" FROM krasch.building_violationssample 
               WHERE CAST("VIOLATION DATE" AS date) > '2014-04-01' 
               AND NOT "LOCATION" IS NULL 
               AND "INSPECTION STATUS" = 'FAILED' 
               GROUP BY "LOCATION"; """

failed_since_april = query_db(sql_query)

# the location is a good index for our dataframe; setting it as an index also makes merging dataframes easier later on
failed_since_april = failed_since_april.set_index("LOCATION")
df.head()

,count
LOCATION,
"(41.70218602130889,-87.62680358268749)",1
"(41.82370467512668,-87.67128029275361)",1
"(41.92428830284015,-87.75087656974485)",1
"(41.99813141435855,-87.75705153629221)",1
"(41.942591613793674,-87.72254386304228)",1


### What if we want failed inspections in time range?

In [3]:
def failures_in_timerange(start_date, end_date):
    # make sure that start date and end date are formatted correctly (so that postgres can understand)
    start_date = start_date.strftime('%Y-%m-%d')
    end_date = end_date.strftime('%Y-%m-%d')
    
    # make a query for this time range
    sql_query = """SELECT COUNT(*), "LOCATION" FROM krasch.building_violationssample 
                   WHERE CAST("VIOLATION DATE" AS date) >= '{start}' 
                   AND CAST("VIOLATION DATE" AS date) <'{end}' 
                   AND NOT "LOCATION" IS NULL 
                   AND "INSPECTION STATUS" = 'FAILED' 
                   GROUP BY "LOCATION"; """.format(start=start_date, end=end_date)
    
    # query and set location as index
    failed_df = query_db(sql_query)
    failed_df = failed_df.set_index("LOCATION")
    
    return failed_df["count"]


### Let's try out our new function to get violations in the 30 days until today

In [4]:
range_length = timedelta(days=30)
failures_30days = failures_in_timerange(date.today() - range_length, date.today())
failures_30days.name = "Failures30"

### And in the last 90 days

In [5]:
range_length = timedelta(days=90)
failures_90days = failures_in_timerange(date.today() - range_length, date.today())
failures_90days.name = "Failures90"
failures_90days.head()

LOCATION
(41.782223140005044,-87.6126196710642)     1
(41.78306113766131,-87.68359886966648)     1
(41.76067033194391,-87.67357109982991)     1
(41.78677993100218,-87.65428695708154)     1
(41.911578609818896,-87.71026095198513)    1
Name: Failures90, dtype: int64